In [1]:
import random
import ipyvuetify as v
from traitlets import Int, List, HasTraits, TraitError, observe, validate
import pandas as pd
import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot

# Project

In [2]:
import wdwrap
import wdwrap.ui
from wdwrap.ui.vuetify import EnumSelect, PandasDataFrame
from wdwrap.io import Reader_ravespan_rv, Reader_hjd_mag

In [3]:
project = wdwrap.ui.project.Project()

## Curves List

In [4]:
TEST = None
def list_set_op(lst, element, add):
    if add:
        return list(set(lst) | {element})
    else:  # subtract
        return list(set(lst) - {element})

class EnumSelect(v.Select):
    """ Class for comobobox items"""

    def __init__(self, obj, prop, **kwargs) -> None:
        self.trait = obj.class_traits()[prop]
        self.obj = obj
        self.prop = prop
        super().__init__(items=self.select_items(), v_model=getattr(obj, prop), **kwargs)
    
    def select_items(self):
        return [{'text': f'{k} {v}', 'value': k} for k, v in self.trait.values.items()]
    
    @observe('v_model')  
    def handler_v_model(self, change):
        setattr(self.obj, self.prop, change['new'])
#         print(f'New val {change["new"]}')

class NumberInput(v.TextField):
    def __init__(self, obj, prop, **kwargs) -> None:
        self.obj = obj
        self.prop = prop
        super().__init__(v_model=getattr(obj, prop), **kwargs)
    
    @observe('v_model')  
    def handler_v_model(self, change):
        setattr(self.obj, self.prop, change['new'])
#         print(f'New val {change["new"]}')

class BoolInput(v.Checkbox):
    def __init__(self, obj, prop, **kwargs) -> None:
        self.obj = obj
        self.prop = prop
        super().__init__(v_model=getattr(obj, prop), **kwargs)
    
    @observe('v_model')  
    def handler_v_model(self, change):
        setattr(self.obj, self.prop, change['new'])
#         print(f'New val {change["new"]}')

def gen_controls(collection, include=None, exclude=None, colskw={}, **kwargs):
    controls = []
    for p in collection.params:
        if include is not None and p.name not in include:
            continue
        if exclude is not None and p.name in exclude:
            continue
        kwargs['label'] = p.name
        kwargs['hint'] = p.pclass.help_str
        if p.pclass.help_val:
            va = EnumSelect(p, 'val', **kwargs)
        elif isinstance(p, wdwrap.param.FloatParameter):
            va = NumberInput(p, 'ui_val', **kwargs)
        else:
            va = NumberInput(p, 'ui_val', **kwargs)
#         controls.append(v.Col(children=[v.Card(children=[p.name, va])]))
        controls.append(v.Col(children=[va], **colskw))
#         controls.append(va)
    return controls


class WdControlsContainer(v.Container):
    def __init__(self, parent, rows, params, **kwargs) -> None:
        self.parent = parent
        self.params = params
        self.rows = rows
        super().__init__(children=self.make_children(), **kwargs)
        
    def make_children(self):
        children = []
        for r in rows:
            children.append(self.make_wdpar_row(first=True, **r))
        return children
            
    def make_wdpar_row(self, include=None, exclude=None):
        return v.Row(wrap=True, dense=True, children=self.gen_controls(
            self.params, colskw={'cols':False},
            include=include, exclude=exclude)) 
    
    @staticmethod
    def gen_controls(collection, include=None, exclude=None, colskw={}, **kwargs):
        controls = []
        for p in collection.params:
            if include is not None and p.name not in include:
                continue
            if exclude is not None and p.name in exclude:
                continue
            kwargs['label'] = p.name
            kwargs['hint'] = p.pclass.help_str
            if p.pclass.help_val:
                va = EnumSelect(p, 'val', **kwargs)
            elif isinstance(p, wdwrap.param.FloatParameter):
                va = NumberInput(p, 'ui_val', **kwargs)
            else:
                va = NumberInput(p, 'ui_val', **kwargs)
    #         controls.append(v.Col(children=[v.Card(children=[p.name, va])]))
            controls.append(v.Col(children=[va], **colskw))
    #         controls.append(va)
        return controls





class CurveExtPanel(v.ExpansionPanel):
    curv_par_ln1 = ['IBAND', 'HLUM', 'CLUM', 'EL3', 'WL']

    def __init__(self, curve, project, parent, **kwargs) -> None:
        self.color = f'#{random.randint(0, 0xffffff):06x}44'
        self.project = project
        self.parent = parent
        self.curve = curve
        self.controlcol_kw = {'cols': False}
        self.control_kw = {'outlined': False, 'dense': True}
        self.element_observed = v.Row()
        self.avatar = v.Avatar(children=[self.get_atavartext()], 
                         color=self.color, size=36)
        self.filename = v.Html(tag='div', children=self.make_filename())
        super().__init__(children=self.make_children(), **kwargs)
        self.curve.observe(lambda ch: self.on_curve_plot_change(ch), 'plot')
        self.curve.observe(lambda ch: self.on_curve_fit_change(ch), 'fit')
        self.curve.obs_values.observe(lambda ch: self.on_observed_df_change(ch), 'dataframe')
        self.curve.obs_values.observe(lambda ch: self.on_observed_df_dataversion_change(ch), 'df_version')
        self.curve.obs_values.observe(lambda ch: self.on_observed_filename_change(ch), 'filename')
        
    @property
    def is_rv(self):
        return isinstance(self.curve, wdwrap.ui.curves.VelocCurve)

    def get_atavartext(self):
        if self.is_rv:
            return 'RV'
        else:
            iband = self.curve.wdparams['IBAND']
            if iband is not None:
                def on_iband_changed(chng):
                    self.avatar.children = [self.get_atavartext()]
                iband.observe(on_iband_changed, 'val', 'change')
                return iband.pclass.help_val[iband.val].split()[0]
            
    
    def make_children(self):
        del_btn = v.Btn(small=True, icon=True, color='red', children=[
                    v.Icon(small=True, dense=True, children=['mdi-close'])])
        del_btn.on_event('click', lambda *args: self.on_delete())
        upload_btn = v.Btn(small=True, color='primary', children=['Upload...'])
        upload_btn.on_event('click', lambda *args: self.on_upload())
        self.element_observed.children=self.make_observed()
        

        hdr = v.ExpansionPanelHeader(children=[
            v.Row(children=[
                v.Col(children=[self.avatar]),
                v.Col(children=[self.filename]),
                v.Spacer(),
                self.make_buttons(),
                del_btn,
                ]),
            ])
        cnt = v.ExpansionPanelContent(children=[
            v.Container(fluid=True, children=[
                self.make_wdpar_row(first=True),
                self.make_wdpar_row(first=False),
            ]),
            v.Container(fluid=True, children=[
                v.Row(children=[
                    v.Col(children=['Observed']),
                    upload_btn,
                ]),
                self.element_observed,
            ])
        ])        
        return [hdr, cnt]
    
    def make_observed(self):
        obs = self.curve.obs_values
        if obs.empty():
            return []
        controls = [
            NumberInput(obs.phaser, 'period', label='period', hint='[d] can be empty'),
            NumberInput(obs.phaser, 'hjd0', label='HJD 0', hint='shifts hjd'),
            NumberInput(obs.phaser, 'delta', label='phase delta', hint='in days, shifts phase'),
            BoolInput(obs.resampler, 'active', label='binning'),
            NumberInput(obs.resampler, 'k', label='# of bins', hint='equal width bins'),
        ]
        return [v.Col(children=[c]) for c in controls]
        
    
    def make_buttons(self):
        model = []
        model = list_set_op(model, 'plot', self.curve.plot)
        model = list_set_op(model, 'fit', self.curve.fit)
        self.buttons = v.BtnToggle(multiple=True, dense=True, group=True, color='primary', 
            v_model=model,
            children=[
                self.make_button('plot', icon='mdi-eye-outline', tooltip='Show on plot'),
                self.make_button('fit', icon='mdi-vector-radius', tooltip='Active in fitting'),
            ])
        self.buttons.observe(lambda ch: self.on_button_change(ch), 'v_model')
        return self.buttons
    
    def make_button(self, val, icon, tooltip=None, **kwargs):
        if tooltip:
            kwargs['slot'] = 'activator'
        btn = v.Btn(value=val, small=True, children=[
            v.Icon(small=True, dense=True, children=[icon])], **kwargs)
        if tooltip:
            return v.Tooltip(children=[btn])
        else:
            return btn

    def make_wdpar_row(self, first: bool):
        return v.Row(wrap=True, dense=True, children=self.gen_controls(
            self.curve.wdparams, colskw={'cols':False},
            include=self.curv_par_ln1 if first else None, exclude=None if first else self.curv_par_ln1, 
            **self.control_kw)) 
    
    def make_filename(self, filename=None):
        if filename is None:
            filename = self.curve.obs_values.filename
        if filename:
            return [v.Chip(children=[filename])]
        else:
            return []

    def on_button_change(self, change):
        self.curve.plot = 'plot' in self.buttons.v_model
        self.curve.fit  = 'fit'  in self.buttons.v_model
            
    def on_delete(self):
        self.project.delete_curve(self.curve)
        
    def on_curve_plot_change(self, change):
        self.buttons.v_model = list_set_op(self.buttons.v_model, 'plot', change['new'])

    def on_curve_fit_change(self, change):
        self.buttons.v_model = list_set_op(self.buttons.v_model, 'fit', change['new'])
        
    def on_observed_df_change(self, change):
        observed_controls = self.make_observed()
        self.element_observed.children = observed_controls

    def on_observed_df_dataversion_change(self, change):
        pass
    
    def on_observed_filename_change(self, change):
        new = change['new']
        self.filename.children = self.make_filename(new)
        
    def on_upload(self):
        self.parent.parent.parent.upload_dialog(self)
        
    @staticmethod
    def gen_controls(collection, include=None, exclude=None, colskw={}, **kwargs):
        controls = []
        for p in collection.params:
            if include is not None and p.name not in include:
                continue
            if exclude is not None and p.name in exclude:
                continue
            kwargs['label'] = p.name
            kwargs['hint'] = p.pclass.help_str
            if p.pclass.help_val:
                va = EnumSelect(p, 'val', **kwargs)
            elif isinstance(p, wdwrap.param.FloatParameter):
                va = NumberInput(p, 'ui_val', **kwargs)
            else:
                va = NumberInput(p, 'ui_val', **kwargs)
    #         controls.append(v.Col(children=[v.Card(children=[p.name, va])]))
            controls.append(v.Col(children=[va], **colskw))
    #         controls.append(va)
        return controls

class CurvesPanels(v.ExpansionPanels):

    def __init__(self, project, parent, **kwargs) -> None:
        self.project = project
        self.parent = parent
        self.panels = {}
        super().__init__(children=self.make_children(), multiple=True, accordion=True, **kwargs)
        
        project.observe(lambda change: self.on_curve_mutation(change, rv=True), 'veloc_curves_len')
        project.observe(lambda change: self.on_curve_mutation(change, rv=False), 'light_curves_len')
        
    def on_curve_mutation(self, change, rv):
        print (f'Time of change rv={rv} {change["old"]} => {change["new"]}')
        self.children = self.make_children()

        
    def make_children(self):
        rvpanels = [self.make_panel(c) for c in project.veloc_curves]
        ltpanels = [self.make_panel(c) for c in project.light_curves]
        panels = rvpanels + ltpanels
        self.panels = {id(p.curve): p for  p in panels}
        return rvpanels + ltpanels
    
    def make_panel(self, c):
        # try to reuse
        try:
            p = self.panels[id(c)]
        except LookupError:
            p = CurveExtPanel(c, self.project, parent=self)
        return p
    
    @validate('v_model')
    def openclose_ok(self, proposal):
        p = proposal['value']
        return p
    

#    <v-btn outline color="indigo">Outline Button</v-btn>

class UploadDialog(v.Dialog):
    def __init__(self, project, parent, **kwargs) -> None:
        self.project = project
        self.parent = parent
        self.icon = ''
        self.curve = None
        self.datatable = PandasDataFrame(caption='Data')
        self.filename = ''
        self.filename_tag = v.Html(tag='div', children=['Filename: <no file>'])
        super().__init__(children=self.make_children(), v_model=False, scrollable=True, **kwargs)
        
    @property
    def rv(self):
        try: 
            return self.curve.is_rv()
        except AttributeError:
            return False
        
    def make_children(self):
#         cancelbtn = v.Btn(text=True, color="red darken-1", children=['Cancel'])
        cancelbtn = v.Btn(icon=True, dark=True,  children=[v.Icon(children=['mdi-close'])])
        savebtn = v.Btn(text=True, dark=True, children=['Save']) # color="green darken-1"
        cancelbtn.on_event('click', lambda *args: self.cancel())
        savebtn.on_event('click', lambda *args: self.save())
        return [
            v.Card(children=[
                v.Toolbar(dark=True, color='primary', children=[
                    cancelbtn, self.icon,
                    v.ToolbarTitle(
                        children=[f'Upload observed {"velocity curves" if self.rv else "light curve"}']),
                    v.Spacer(), 
                    v.ToolbarItems(children=[savebtn])
                ]),
                v.CardText(style="height: 300px;", children = self.make_text() + [
                    self.make_sections(),
                    self.datatable])
            ])
        ]

    def make_sections(self):
        s = v.Row(justify='start', align='center', children=[
            self.make_columns_hint(), v.Spacer(),
            self.make_uploader(), v.Spacer(),
            self.make_filename(),
        ])
        return v.CardText(children = [s])
        
    def make_filename(self):
        return self.filename_tag # v.Card(children=[self.filename_tag])
        
    def make_uploader(self):
        uploader = widgets.FileUpload(
            description='Select File',
            icon='plus',
            accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=False  # True to accept multiple files upload else False
        )
        uploader.observe(lambda change: self.on_file_uploaded(change), 'value')
        return v.Chip(label=True, outlined=True, children=[uploader], color='blue')

    def make_columns_hint(self):
        if self.rv:
            columns = ['HJD', 'RV1', 'RV1 error', 'RV2', 'RV2 error']
        else:
            columns = ['HJD', 'mag', 'mag error [optional]']
        html = v.Html(tag='ol', children=[v.Html(tag='li', children=[txt]) for txt in columns])
        return html # v.Card(children=[html])
        
    def make_text(self):
        children=[
            'Currently, the imported files format is fixed.', v.Html(tag='br'),
            'File is expected to be text file, with white space separated columns. '\
            'An empy lines and lines stated with # sign are skipped.', v.Html(tag='br'),
            'The following colums are expected: (additional columns will be ignored)', v.Html(tag='br'),
        ]
        return children
    
    def on_file_uploaded(self, change):
        new = change['new']
        filename, file = list(new.items())[0]
        self.filename = filename
        self.filename_tag.children = [f'Filename: {filename}']
        if self.rv:
            reader = Reader_ravespan_rv()
        else:
            reader = Reader_hjd_mag()
        reader.open_bytes(file['content'], filename)
        self.datatable.df = reader.df
    
        
    def cancel(self):
        self.close()
        
    def save(self):
        self.curve.obs_values.set_df(self.datatable.df)
        self.curve.obs_values.filename = self.filename
        self.close()
        
    def show(self, curve, icon=''):
        self.curve = curve
        self.icon = icon
        self.filename = ''
        self.filename_tag.children = []
        self.children = self.make_children()
        self.datatable.df = pd.DataFrame()
        self.v_model = True
        
    def close(self):
        self.v_model = False

class CurvesCard(v.Card):
    def __init__(self, project, parent=None, **kwargs) -> None:
        self.project = project
        self.parent = parent
        super().__init__(children=self.make_children(), **kwargs)
        
    def make_children(self):
        new_rv_btn = self.make_button('RV', 'mdi-plus', outlined=True, small=True, color='primary')
        new_lt_btn = self.make_button('Light', 'mdi-plus', outlined=True, small=True, color='primary')
        new_rv_btn.on_event('click', lambda *args: self.project.new_curve(rv=True))
        new_lt_btn.on_event('click', lambda *args: self.project.new_curve(rv=False))
        title = v.CardTitle(children=[
            v.Row(children=['Curves', v.Spacer(), new_rv_btn, new_lt_btn])])

        panels = CurvesPanels(project, parent=self, v_model=[0,1])
        return [title, panels]
    
    def make_button(self, text, icon=None, kw_icon={}, **kwargs):
        if icon is not None:
            children = [v.Icon(left=True, small=True, children=[icon], **kw_icon), text]
        else:
            children = [text]
        return v.Btn(children=children, **kwargs)

class LcCard(v.Card):
    def __init__(self, project, parent=None, **kwargs) -> None:
        self.project = project
        self.parent = parent
        super().__init__(children=self.make_children(), **kwargs)
        
    def make_children(self):
        title = v.CardTitle(children=[
            v.Row(children=['WD LC parameters']),
            
        ])
# dupa
#         panels = CurvesPanels(project, parent=self, v_model=[0,1])
        return [title, self.make_parameters(), 
                self.make_charts(), self.make_charts()]
    
    def make_parameters(self):
        return ''
    
    def make_charts(self):
        n = 200

        x = np.linspace(0.0, 10.0, n)
        y = np.cumsum(np.random.randn(n)*10).astype(int)

        fig = plt.figure( title='Histogram')
        np.random.seed(0)
        hist = plt.hist(y, bins=25)
        hist.scales['sample'].min = float(y.min())
        hist.scales['sample'].max = float(y.max())
        fig.layout.width = 'auto'
        fig.layout.height = 'auto'
#         fig.layout.min_height = '300px' # so it shows nicely in the notebook
        return fig
    
        

        
class AppArea(v.Layout):

    def __init__(self, project, **kwargs) -> None:
        self.project = project
        self.upload_dlg = UploadDialog(project, parent=self)
        super().__init__(children=self.make_children(), _metadata={'mount_id': 'content-main'}, 
                        )#row=True, **kwargs)
        
    def make_children(self):
        return [
            self.upload_dlg,
            v.Flex(xs12=True, lg6=True, xl4=True, children=[CurvesCard(self.project, parent=self)]),
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                v.Tabs(children=[
                    v.Tab(children=['LC']),
                    v.Tab(children=['Markov CMC']),
                    v.TabItem(children=[LcCard(self.project, parent=self)]),
                    v.TabItem(children=['Macrcov Chain Monte Carlo goes here']),
                ])
            ]),
        ]
    
    def upload_dialog(self, curve_panel: CurveExtPanel):
        self.upload_dlg.show(curve = curve_panel.curve, icon=curve_panel.avatar)


In [5]:
app = AppArea(project)
app

AppArea(children=[UploadDialog(children=[Card(children=[Toolbar(children=[Btn(children=[Icon(children=['mdi-cl…

In [6]:
c = project.veloc_curves[0]
o = c.obs_values.df

In [7]:
#c.obs_values.dataframe = pd.DataFrame()

In [8]:
c.obs_values.traits()['dataframe']
cdt = c.obs_values.traits()['dataframe']

In [9]:
c.obs_values._trait_notifiers['dataframe']['change'][0]

<function __main__.CurveExtPanel.__init__.<locals>.<lambda>(ch)>

In [10]:
c.obs_values.filename

''

In [11]:
c.obs_values._trait_notifiers

{'dataframe': {'change': [<function __main__.CurveExtPanel.__init__.<locals>.<lambda>(ch)>]},
 'df_version': {'change': [<function __main__.CurveExtPanel.__init__.<locals>.<lambda>(ch)>]},
 'filename': {'change': [<function __main__.CurveExtPanel.__init__.<locals>.<lambda>(ch)>]}}

In [12]:
c.obs_values.df_version = 8

In [13]:
def px(change):
    print (change['new'])
c.obs_values.observe(px, 'df_version')

In [14]:
curve_gen_rows = [
    v.Row(children=gen_controls(c.wdparams, include=curv_par_ln1, 
                                outlined=True, dense=True, colskw={'cols':False}), 
          wrap=True, dense=True),
    v.Row(children=gen_controls(c.wdparams, exclude=curv_par_ln1, 
                                outlined=True, dense=True, colskw={'cols':False}), 
          wrap=True, dense=True, row=False),
]

curve_gen_par = v.Container(children=curve_gen_rows, fluid=True)


NameError: name 'curv_par_ln1' is not defined

In [ ]:
u = v.Layout(children=[
    v.Btn(color='primary', children=[
        v.Icon(left=True, children=['fingerprint']),
        'Icon left'
    ]),
    v.Btn(color='primary', children=[
        'Icon right',
        v.Icon(right=True, children=['fingerprint']),
    ]),
    v.Tooltip(children=[
        v.Btn(slot='activator', color='primary', children=[
            'tooltip dupa'
        ]),
        'Insert tooltip text here'
    ])
])
display(u)

In [ ]:
c.wdparams.param_dict['IBAND'].val

In [ ]:
' '.join(c.wdparams.param_dict), c.wdparams.param_dict['IBAND'].val

In [ ]:


rvpanels = [
    v.ExpansionPanel(children=[
        v.ExpansionPanelHeader (children=[v.Row(children=[v.Avatar(children=['RV'], color='pink', size=36)])]),
        v.ExpansionPanelContent(children=[curve_gen_rows]),
    ]) for c in project.veloc_curves
]
lpanels = [
    v.ExpansionPanel(children=[
        v.ExpansionPanelHeader (children=[str(c.wdparams.param_dict['IBAND'].val)]),
        v.ExpansionPanelContent(children=['a','b']),
    ]) for c in project.veloc_curves
]

clist = v.ExpansionPanels(children=rvpanels + lpanels, accordion=True)

display(clist)

In [ ]:
xxx = v.Avatar(children=['RV'], color='FF345644', size=36)
xxx

In [ ]:
f'#{random.randint(0, 0xffffff):06x}44'

In [ ]:

class EnumSelect(v.Select):
    """ Class for comobobox items"""

    def __init__(self, obj, prop, **kwargs) -> None:
        self.trait = obj.class_traits()[prop]
        self.obj = obj
        self.prop = prop
        super().__init__(items=self.select_items(), v_model=getattr(obj, prop), **kwargs)
    
    def select_items(self):
        return [{'text': v, 'value': k} for k, v in self.trait.values.items()]
    
    @observe('v_model')  
    def handler_v_model(self, change):
        setattr(self.obj, self.prop, change['new'])
        print(f'New val {change["new"]}')

class NumberInput(v.TextField):
    def __init__(self, obj, prop, **kwargs) -> None:
        self.trait = obj.class_traits()[prop]
        self.obj = obj
        self.prop = prop
        super().__init__(v_model=getattr(obj, prop), **kwargs)
    
    @observe('v_model')  
    def handler_v_model(self, change):
        setattr(self.obj, self.prop, change['new'])
        print(f'New val {change["new"]}')
    

In [ ]:
i = EnumSelect(xx, 'val', chips=False, filled=True, outlined=True, dense=True)
j = NumberInput(xx, 'val', chips=False, filled=True, outlined=True, dense=True)

In [ ]:
i.v_model

In [ ]:
v.Container(children=[v.Row(children=[i,j])])

In [ ]:
xx.val

In [ ]:
v.Select(items=[{'text':'dupa', 'value':12}])

In [ ]:
xx.pclass.help_val

# First histogram plot

In [ ]:
import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot


n = 200

x = np.linspace(0.0, 10.0, n)
y = np.cumsum(np.random.randn(n)*10).astype(int)

fig = plt.figure( title='Histogram')
np.random.seed(0)
hist = plt.hist(y, bins=25)
hist.scales['sample'].min = float(y.min())
hist.scales['sample'].max = float(y.max())
fig.layout.width = 'auto'
fig.layout.height = 'auto'
fig.layout.min_height = '300px' # so it shows nicely in the notebook
fig

In [ ]:
slider = v.Slider(thumb_label='always', class_="px-4", v_model=30)
widgets.link((slider, 'v_model'), (hist, 'bins'))
slider

### Curves

In [ ]:
from wdwrap.ui.curveslist import LcCurvesList, RvCurvesList
from wdwrap.ui.uplader import LcUploader, RvUploader

In [ ]:
lcs = LcCurvesList()

In [ ]:
vAddLC = LcUploader()
vAddRV = RvUploader()
display(vAddLC.widget, vAddRV.widget, )

In [ ]:
list(vAddLC.widget.value.values())[0]['metadata']['name']

In [ ]:
lcs.add(file=vAddLC.filename(), data=vAddLC.to_dataframe())

In [ ]:
lcs.df[2, 'active'] = False

In [ ]:
lcs.df

In [ ]:
xxx = False

In [ ]:
vLCs = v.DataTable(items=lcs.items_json(), headers=lcs.headers_json(), 
                   items_per_page=20, caption='Light Curves', dense=True, disable_pagination=True,
                   v_slots=[
                       {'name': 'item.active', 'variable':'x','children': [v.Checkbox(label='x.value')]},
                       {'name': 'item.plot', 'variable':'y', 'children': [v.SimpleCheckbox(v_model='y')]},
                   ]
                  )
vLCs

In [ ]:
v.Label(caption='sdsd')

In [ ]:
vLCs = v.DataTable(items=[{'hjd':1212, 'mag':12.3}], 
                   headers=[{ 'text': 'HJD', 'value': 'hjd' },{ 'text': 'mag', 'value': 'mag' }], 
                   items_per_page=20)
vLCs

In [ ]:

addLC = widgets.FileUpload(
    description='Observed LC',
    icon='plus',
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
addRV = widgets.FileUpload(
    description='Observed RV',
    icon='plus',
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
button = widgets.Button(description="Click Me!")
display(button)
display(addLC) 
display(addRV)

In [ ]:
xx = ((list(addLC.value.values())[0]['content']))

In [ ]:
from io import StringIO, BytesIO
zzo = BytesIO(xx)

In [ ]:
pd.read_table(zzo, names=['hjd', 'mag'], index_col=0, usecols=[0,1], delim_whitespace=True)

In [ ]:
output = widgets.Output()
def pt(u): 
    with output:
        print(u)
def on_button_clicked(b):
    with output:
        print(f"Button clicked.{b}")
display(output)
addLC.observe(pt)
button.on_click(on_button_clicked)

In [ ]:
on_button_clicked(5)
pt(1)

In [ ]:
uploaded_file["content"]

In [ ]:
ul.keys

In [ ]:
ul.icon

supernew

In [ ]:
v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['dupa1']),
    v.Tab(children=['Tab2']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig, slider, ul
            ]),
#             v.Flex(xs12=True, lg6=True, xl4=True, children=[
#                 figHist2, sliderHist2
#             ]),
#             v.Flex(xs12=True, xl4=True, children=[
#                 fig2
#             ]),
        ])
    ]),
    v.TabItem(children=[
        v.Container(children=['Lorum ipsum'])
    ])
])